## Proyecto 1. Analisis de contratos y proyectos de inversión del gobierno de Colombia.

**Importar funciones de Spark**

*Funciones F*: colección de funciones matemáticas y de procesamiento de datos para DataFrames.

*Funciones Window*: Permite hacer cáculos sobre un conjunto de filas sin colapsar el DataFrame.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

**Ejercicios**

In [0]:
# ========================================
# EJERCICIO 1: LECTURA OPTIMIZADA DE DATOS
# ========================================

# Rutas de los datasets
secop_path = "wasbs://sid@uniandesyjt.blob.core.windows.net/secop"
bpin_path = "wasbs://sid@uniandesyjt.blob.core.windows.net/bpin"

# LECTURA OPTIMIZADA DE SECOP (Contratos)
df_secop = spark.read \
    .format("json") \
    .option("multiline", "true") \
    .option("samplingRatio", "0.1") \
    .load(secop_path)

# LECTURA OPTIMIZADA DE BPIN (Proyectos)
df_bpin = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(bpin_path)

**Justificación de cada función utilizada en la lectura de información de SECOP**

format("json"): Los archivos de contratos están en formato JSON comprimido (.json.gz)

multiline=true: Los contratos tienen estructura compleja que puede ocupar múltiples líneas. Sin esto, Spark fallaría al leer contratos con descripciones largas

samplingRatio=0.1: Analiza solo 10% de los archivos para inferir la estructura.

**Justificación de cada función utilizada en la lectura de información de SECOP**

format("csv"): Los datos de proyectos están en formato CSV (.csv.gz)

header=true: La primera fila contiene nombres de columnas (Bpin, NombreProyecto, etc.), no datos

inferSchema=true: Detecta automáticamente los tipos de datos (números vs texto). Sin esto, todas las columnas se leen como texto y las operaciones matemáticas fallarían

In [0]:
# ========================================
# EJERCICIO 2: TOP 10 PROVEEDORES 2024 
# ========================================

print("\n" + "=" * 50)
print("EJERCICIO 2: TOP 10 PROVEEDORES 2024")
print("=" * 50)

# Filtrar contratos de 2024 y obtener top proveedores por valor
top_proveedores_2024 = df_secop \
    .filter(F.col("anno_firma") == 2024) \
    .groupBy("proveedor_adjudicado") \
    .agg(F.sum("valor_del_contrato").alias("valor_total_contratos")) \
    .orderBy(F.desc("valor_total_contratos")) \
    .limit(10)

# Convertir a formato más legible para reporte
resultado_ej2 = top_proveedores_2024.collect()
for i, row in enumerate(resultado_ej2, 1):
    valor_formateado = f"${row['valor_total_contratos']:,.0f}"
    print(f"{i:2}. {row['proveedor_adjudicado'][:50]:<50} {valor_formateado:>20}")



EJERCICIO 2: TOP 10 PROVEEDORES 2024
 1. MARIA SORELY GRISALES / CREACIONES BJ                $2,115,156,861,243
 2. UNIVERSIDAD DE ANTIOQUIA                             $1,856,417,293,206
 3. INSTITUTO PARA EL DESARROLLO DE ANTIOQUIA IDEA       $1,703,797,668,980
 4. FINANCIERA DE DESARROLLO TERRITORIAL S.A.            $1,541,604,758,133
 5. INSTITUTO TECNOLOGICO METROPOLITANO                  $1,152,177,220,033
 6. BANCOLOMBIA                                          $1,022,475,569,903
 7. SOCIEDAD DE ACTIVOS ESPECIALES SAS                   $1,000,549,529,167
 8. ESU                                                    $870,387,131,033
 9. correagro                                              $841,630,245,974
10. UNIDAD PARA LAS VÍCTIMAS - FONDO PARA LA REPARACIÓ     $710,000,000,000


In [0]:
# ========================================
# EJERCICIO 3: PROYECTOS CON MAYOR VALOR SIN PAGAR
# ========================================

print("\n" + "=" * 50)
print("EJERCICIO 3: PROYECTOS CON MAYOR VALOR SIN PAGAR")
print("=" * 50)

# Unir SECOP con BPIN usando la columna común (bpin)
# Calcular valor sin pagar = ValorTotalProyecto - ValorPagoProyecto
proyectos_deuda = df_bpin \
    .withColumn("valor_sin_pagar", 
                F.col("ValorTotalProyecto") - F.col("ValorPagoProyecto")) \
    .filter(F.col("valor_sin_pagar") > 0) \
    .orderBy(F.desc("valor_sin_pagar")) \
    .limit(10)

# Convertir a formato más legible para reporte
resultado_ej3 = proyectos_deuda.collect()
for i, row in enumerate(resultado_ej3, 1):
    valor_formateado = f"${row['valor_sin_pagar']:,.0f}"
    bpin_code = row['Bpin']
    proyecto_nombre = row['NombreProyecto'][:80] + "..." if len(row['NombreProyecto']) > 80 else row['NombreProyecto']
    print(f"{i:2}. BPIN: {bpin_code}")
    print(f"    {proyecto_nombre}")
    print(f"    Valor sin pagar: {valor_formateado}")
    print()


EJERCICIO 3: PROYECTOS CON MAYOR VALOR SIN PAGAR
 1. BPIN: 2022052090029
    Mejoramiento De Procesos Agropecuarios En El Municipio De  Concordia
    Valor sin pagar: $186,666,674,666,667

 2. BPIN: 202400000000008
    Construcción De Los Proyectos Sistemas Integrados De Transporte Masivo Sitm Dist...
    Valor sin pagar: $92,842,095,357,321

 3. BPIN: 202300000000455
    Implementación De La Política De Gratuidad Y Estrategias Para La Financiación De...
    Valor sin pagar: $73,841,716,026,232

 4. BPIN: 2018011000720
    Apoyo A Proyectos De Inversión A Nivel  Nacional
    Valor sin pagar: $36,718,368,795,103

 5. BPIN: 202300000000204
    Implementación De Transferencias Monetarias Para Población En Situación De Pobre...
    Valor sin pagar: $28,908,051,313,453

 6. BPIN: 202500000002103
    Fortalecimiento Al Proceso De Aseguramiento Y Atenciòn De Pqrs Prestación De Ser...
    Valor sin pagar: $28,716,343,392,272

 7. BPIN: 2017733490012
    Fortalecimiento De Salud En Aseguramien

In [0]:
# ========================================
# EJERCICIO 4: RANKING ANUAL DE PROVEEDORES
# ========================================

print("\n" + "=" * 50)
print("EJERCICIO 4: RANKING ANUAL DE PROVEEDORES")
print("=" * 50)

# Usar funciones de ventana para ranking anual
# 1. Calcular valor total por proveedor por año
# 2. Asignar ranking dentro de cada año
# 3. Filtrar solo los top 5 de cada año

from pyspark.sql.window import Window

# Definir ventana para particionar por año y ordenar por valor total
ventana_anual = Window.partitionBy("anno_firma").orderBy(F.desc("valor_total"))

ranking_anual = df_secop \
    .groupBy("anno_firma", "proveedor_adjudicado") \
    .agg(F.sum("valor_del_contrato").alias("valor_total")) \
    .withColumn("ranking", F.row_number().over(ventana_anual)) \
    .filter(F.col("ranking") <= 5) \
    .orderBy("anno_firma", "ranking")

# Convertir a formato más legible para reporte
resultado_ej4 = ranking_anual.collect()

año_actual = None
for row in resultado_ej4:
    if row['anno_firma'] != año_actual:
        año_actual = row['anno_firma']
        print(f"\n🗓️  AÑO {año_actual}:")
        print("=" * 60)
    
    valor_formateado = f"${row['valor_total']:,.0f}"
    proveedor = row['proveedor_adjudicado'][:50] + "..." if len(row['proveedor_adjudicado']) > 50 else row['proveedor_adjudicado']
    print(f"   {row['ranking']}. {proveedor:<50} {valor_formateado:>20}")

print("\n✓ Ejercicio 4 completado: Ranking anual de proveedores calculado")


EJERCICIO 4: RANKING ANUAL DE PROVEEDORES

🗓️  AÑO 2015:
   1. UNIÓN TEMPORAL IDOM-SILO                                   $563,760,000
   2. CORPORACIÓN ECOVERSA                                       $520,000,000
   3. UT MAPFRE - PREVISORA                                      $145,120,101
   4. GRUPO SPIRA SAS                                            $119,828,000
   5. AXA COLPATRIA SEGUROS S.A                                   $98,600,000

🗓️  AÑO 2016:
   1. KAPITAL                                                $151,889,229,120
   2. John Alexander Ardila Evan                              $27,000,000,000
   3. LUZ ESTELA ROJAS QUINTERO                               $18,000,000,000
   4. UNION TEMPORAL GENERALI COLOMBIA SEGUROS GENERALES...      $17,958,407,612
   5. AGUILA DE ORO DE COLOMBIA LTDA                          $14,031,229,446

🗓️  AÑO 2017:
   1. FESU                                                   $438,187,721,616
   2. COLFONDOS SA                                 

In [0]:
# ========================================
# EJERCICIO 5: PROVEEDORES NUEVOS EN 2024
# ========================================

print("\n" + "="*50)
print("EJERCICIO 5: PROVEEDORES QUE FIRMARON EN 2024 PERO NO EN 2020")
print("="*50)

# Proveedores que firmaron en 2024
proveedores_2024 = df_secop \
    .filter(F.col("anno_firma") == 2024) \
    .select("proveedor_adjudicado").distinct()

# Proveedores que firmaron en 2020
proveedores_2020 = df_secop \
    .filter(F.col("anno_firma") == 2020) \
    .select("proveedor_adjudicado").distinct()

# Proveedores nuevos: están en 2024 pero NO en 2020
proveedores_nuevos = proveedores_2024 \
    .join(proveedores_2020, "proveedor_adjudicado", "left_anti")

num_proveedores_nuevos = proveedores_nuevos.count()
print(f"Número de proveedores que firmaron en 2024 pero NO en 2020: {num_proveedores_nuevos}")

proveedores_nuevos.show(20, truncate=False)


EJERCICIO 5: PROVEEDORES QUE FIRMARON EN 2024 PERO NO EN 2020
Número de proveedores que firmaron en 2024 pero NO en 2020: 390664
+-----------------------------------------------------------------------------+
|proveedor_adjudicado                                                         |
+-----------------------------------------------------------------------------+
|SUGAR LEE DILBERT HUFFINGTON                                                 |
|angelica maria rios londoño                                                  |
|Leonor Bedoya Valencia                                                       |
|SANTIAGO VALLEJO                                                             |
|DAMARYS MANTILLA PORRAS                                                      |
|Maritza Yamile Castaño Rincón                                                |
|Ana Mercedes Contreras Beltran                                               |
|GASTROCENTERCentro de Gastroenterología y Endoscopia Digestiva avanza

In [0]:
# ========================================
# EJERCICIO 6: CONTRATOS DE PRESTACIÓN DE SERVICIOS
# ========================================

print("\n" + "="*50)
print("EJERCICIO 6: ANÁLISIS DE CONTRATOS DE PRESTACIÓN DE SERVICIOS")
print("="*50)

# Filtrar contratos que contengan "Prestación de servicios"
contratos_prestacion = df_secop \
    .filter(F.col("objeto_del_contrato").contains("Prestación de servicios"))

# Calcular promedio y mediana aproximada
estadisticas_prestacion = contratos_prestacion.agg(
    F.count("valor_del_contrato").alias("total_contratos"),
    F.avg("valor_del_contrato").alias("valor_promedio"),
    F.expr("approx_percentile(valor_del_contrato, 0.5)").alias("mediana_aproximada")
)

print("ESTADÍSTICAS DE CONTRATOS DE PRESTACIÓN DE SERVICIOS:")
estadisticas_prestacion.show()



EJERCICIO 6: ANÁLISIS DE CONTRATOS DE PRESTACIÓN DE SERVICIOS
ESTADÍSTICAS DE CONTRATOS DE PRESTACIÓN DE SERVICIOS:
+---------------+-------------------+------------------+
|total_contratos|     valor_promedio|mediana_aproximada|
+---------------+-------------------+------------------+
|         415946|4.789700262915619E7|          19000949|
+---------------+-------------------+------------------+



In [0]:
# ========================================
# EJERCICIO 7: AÑO CON MÁS PROVEEDORES
# ========================================

print("\n" + "="*50)
print("EJERCICIO 7: AÑO CON MÁS PROVEEDORES ÚNICOS")
print("="*50)

# Usar approx_count_distinct para optimizar
proveedores_por_año = df_secop \
    .groupBy("anno_firma") \
    .agg(F.approx_count_distinct("proveedor_adjudicado").alias("num_proveedores_aprox")) \
    .orderBy(F.desc("num_proveedores_aprox"))

print("PROVEEDORES ÚNICOS POR AÑO (APROXIMADO):")
proveedores_por_año.show()


EJERCICIO 7: AÑO CON MÁS PROVEEDORES ÚNICOS
PROVEEDORES ÚNICOS POR AÑO (APROXIMADO):
+----------+---------------------+
|anno_firma|num_proveedores_aprox|
+----------+---------------------+
|      2024|               558936|
|      2025|               510578|
|      2023|               505760|
|      2022|               419046|
|      2021|               361515|
|      2020|               240723|
|      2019|               110939|
|      2018|                98952|
|      2017|                17044|
|      2016|                 1212|
|      2015|                   10|
+----------+---------------------+



In [0]:
# ========================================
# EJERCICIO 8: ANÁLISIS DE TEXTO
# ========================================

print("\n" + "="*50)
print("EJERCICIO 8: ANÁLISIS DE TEXTO - PALABRAS EN OBJETOS")
print("="*50)

# Análisis básico de texto en objeto_del_contrato
palabras_frecuentes = df_secop \
    .select(F.explode(F.split(F.lower(F.col("objeto_del_contrato")), " ")).alias("palabra")) \
    .filter(F.length(F.col("palabra")) > 3) \
    .groupBy("palabra") \
    .count() \
    .orderBy(F.desc("count")) \
    .limit(20)

print("TOP 20 PALABRAS MÁS FRECUENTES EN OBJETOS DE CONTRATOS:")
palabras_frecuentes.show(truncate=False)

print("\n" + "="*50)


EJERCICIO 8: ANÁLISIS DE TEXTO - PALABRAS EN OBJETOS
TOP 20 PALABRAS MÁS FRECUENTES EN OBJETOS DE CONTRATOS:
+---------------+-------+
|palabra        |count  |
+---------------+-------+
|servicios      |3913696|
|para           |3736385|
|prestar        |1804825|
|profesionales  |1764496|
|apoyo          |1723151|
|gestión        |1442435|
|prestación     |1416687|
|como           |1197566|
|actividades    |947587 |
|desarrollo     |829370 |
|municipio      |768517 |
|salud          |651030 |
|marco          |568999 |
|proyecto       |550220 |
|procesos       |538345 |
|apoyar         |504502 |
|fortalecimiento|438439 |
|departamento   |435861 |
|gestion        |420018 |
|secretaria     |405750 |
+---------------+-------+


